In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

from keras import backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# use all variables as features

df = pd.read_csv('train.csv')

X = df.drop(['Cover_Type', 'Id'], 1)
y = df['Cover_Type']

X = X.values
X = X.astype('float32')

y = y.values
y = y.astype('float32')

# split into testing and training sets
offset = int(X.shape[0]*0.8)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

In [4]:
X.shape
# see number of features

(15120, 54)

In [18]:
(y_train)

array([5., 5., 2., ..., 6., 5., 5.], dtype=float32)

In [6]:
X_train

array([[2.596e+03, 5.100e+01, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.590e+03, 5.600e+01, 2.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.804e+03, 1.390e+02, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.925e+03, 3.520e+02, 2.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.778e+03, 1.110e+02, 1.500e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.748e+03, 9.800e+01, 1.700e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]], dtype=float32)

In [26]:
# encode class values as integers
# get y_train into form keras can use
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
y_train = np_utils.to_categorical(encoded_Y)

In [ ]:
y_train

In [27]:
y_train.shape

(12096, 7)

In [28]:
# get y_test into form keras can use

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
y_test = np_utils.to_categorical(encoded_Y)

In [90]:
# starting neural network model - using sequential
model = Sequential()
model.add(Dense(64, input_dim=54, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
# i have run this model with various combinations of layers, dropout, changing number in dense
# and looking at different activation functions

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_101 (Dense)            (None, 64)                3520      
_________________________________________________________________
dropout_72 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_102 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_73 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_103 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_74 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_104 (Dense)            (None, 2)                 130       
__________

In [91]:
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=5,
                    verbose=1,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# score is really bad :(

Train on 12096 samples, validate on 3024 samples
Epoch 1/5
12096/12096 [==============================] - 2s 198us/step - loss: 1.9472 - acc: 0.1553 - val_loss: 1.9491 - val_acc: 0.0823
Epoch 2/5
12096/12096 [==============================] - 1s 45us/step - loss: 1.9458 - acc: 0.1575 - val_loss: 1.9517 - val_acc: 0.0823
Epoch 3/5
12096/12096 [==============================] - 1s 45us/step - loss: 1.9455 - acc: 0.1571 - val_loss: 1.9538 - val_acc: 0.0823
Epoch 4/5
12096/12096 [==============================] - 1s 49us/step - loss: 1.9448 - acc: 0.1578 - val_loss: 1.9553 - val_acc: 0.0823
Epoch 5/5
12096/12096 [==============================] - 1s 45us/step - loss: 1.9445 - acc: 0.1582 - val_loss: 1.9564 - val_acc: 0.0823
Test loss: 1.9564343709794303
Test accuracy: 0.08234126984126984


In [115]:
model = Sequential()
model.add(Dense(64, input_dim=54, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_150 (Dense)            (None, 64)                3520      
_________________________________________________________________
dense_151 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_152 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_153 (Dense)            (None, 64)                2112      
_________________________________________________________________
dense_154 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_84 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_155 (Dense)            (None, 7)                 455       
Total para

In [116]:
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=5,
                    verbose=1,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# score is still bad and loss increased

Train on 12096 samples, validate on 3024 samples
Epoch 1/5
12096/12096 [==============================] - 3s 252us/step - loss: 13.8953 - acc: 0.1377 - val_loss: 13.4691 - val_acc: 0.1644
Epoch 2/5
12096/12096 [==============================] - 1s 54us/step - loss: 13.9008 - acc: 0.1376 - val_loss: 13.4691 - val_acc: 0.1644
Epoch 3/5
12096/12096 [==============================] - 1s 55us/step - loss: 13.9008 - acc: 0.1376 - val_loss: 13.4691 - val_acc: 0.1644
Epoch 4/5
12096/12096 [==============================] - 1s 60us/step - loss: 13.7199 - acc: 0.1487 - val_loss: 14.7909 - val_acc: 0.0823
Epoch 5/5
12096/12096 [==============================] - 1s 63us/step - loss: 13.5770 - acc: 0.1577 - val_loss: 14.7909 - val_acc: 0.0823
Test loss: 14.790910801559528
Test accuracy: 0.08234126984126984


In [117]:
# how to get confusion matrix from neural net?
# how to get any id for mislabeled points?

# not sure why mlp nn is not performing well, looked at many different combos, none got above 
# scores of 0.17 boo


In [121]:
model = Sequential()
model.add(Dense(8, input_dim=54, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()
#model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_158 (Dense)            (None, 8)                 440       
_________________________________________________________________
dense_159 (Dense)            (None, 7)                 63        
Total params: 503
Trainable params: 503
Non-trainable params: 0
_________________________________________________________________


In [122]:
history = model.fit(X_train, y_train,
                    batch_size=8,
                    epochs=100,
                    verbose=1,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# i stopped this because it is also pretty bad, tried a simpler mlp nn, no luck

Train on 12096 samples, validate on 3024 samples
Epoch 1/100
12096/12096 [==============================] - 5s 442us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 2/100
12096/12096 [==============================] - 3s 284us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 3/100
12096/12096 [==============================] - 3s 277us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 4/100
12096/12096 [==============================] - 3s 274us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 5/100
12096/12096 [==============================] - 4s 322us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 6/100
12096/12096 [==============================] - 4s 293us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 7/100
12096/12096 [==============================] - 4s 302us/step - loss: 13.7302 - acc: 0.1481 - 

12096/12096 [==============================] - 4s 332us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 59/100
12096/12096 [==============================] - 4s 340us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 60/100
12096/12096 [==============================] - 4s 295us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 61/100
12096/12096 [==============================] - 4s 291us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 62/100
12096/12096 [==============================] - 4s 310us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 63/100
12096/12096 [==============================] - 3s 284us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 64/100
12096/12096 [==============================] - 3s 271us/step - loss: 13.7302 - acc: 0.1481 - val_loss: 14.1566 - val_acc: 0.1217
Epoch 65/100
12096/

KeyboardInterrupt: 

In [123]:
model = Sequential()
model.add(Dense(64, input_dim=54, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train,
                    batch_size=8,
                    epochs=100,
                    verbose=1,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# this model performed slightly better, also 0.20 for a final score, but that is still no great
# especially compared to supervised learning models like random forest, accuray = 0.86

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_160 (Dense)            (None, 64)                3520      
_________________________________________________________________
dense_161 (Dense)            (None, 7)                 455       
Total params: 3,975
Trainable params: 3,975
Non-trainable params: 0
_________________________________________________________________
Train on 12096 samples, validate on 3024 samples
Epoch 1/100
12096/12096 [==============================] - 6s 505us/step - loss: 12.9610 - acc: 0.1946 - val_loss: 14.0257 - val_acc: 0.1296
Epoch 2/100
12096/12096 [==============================] - 3s 270us/step - loss: 12.9973 - acc: 0.1930 - val_loss: 14.1838 - val_acc: 0.1197
Epoch 3/100
12096/12096 [==============================] - 3s 279us/step - loss: 12.9965 - acc: 0.1934 - val_loss: 14.0557 - val_acc: 0.1276
Epoch 4/100
12096/12096 [==============================] - 4s 29

12096/12096 [==============================] - 4s 310us/step - loss: 12.1133 - acc: 0.2474 - val_loss: 13.3181 - val_acc: 0.1736
Epoch 55/100
12096/12096 [==============================] - 4s 317us/step - loss: 11.9162 - acc: 0.2601 - val_loss: 13.1202 - val_acc: 0.1858
Epoch 56/100
12096/12096 [==============================] - 4s 315us/step - loss: 11.9849 - acc: 0.2557 - val_loss: 13.2422 - val_acc: 0.1779
Epoch 57/100
12096/12096 [==============================] - 4s 299us/step - loss: 12.1202 - acc: 0.2474 - val_loss: 13.2746 - val_acc: 0.1756
Epoch 58/100
12096/12096 [==============================] - 4s 291us/step - loss: 12.1111 - acc: 0.2481 - val_loss: 13.1955 - val_acc: 0.1809
Epoch 59/100
12096/12096 [==============================] - 3s 287us/step - loss: 12.2880 - acc: 0.2372 - val_loss: 13.3140 - val_acc: 0.1733
Epoch 60/100
12096/12096 [==============================] - 4s 297us/step - loss: 11.9188 - acc: 0.2598 - val_loss: 13.2233 - val_acc: 0.1792
Epoch 61/100
12096/

In [128]:
# let's see if we only use only 10 features instead of 54
df = pd.read_csv('train.csv')
df10 = df.iloc[:, 1:11]

X = df10
y = df['Cover_Type']

X = X.values
X = X.astype('float32')

y = y.values
y = y.astype('float32')


# we will split up this dataset 'train' further into training and testing sets
offset = int(X.shape[0]*0.8)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
y_train = np_utils.to_categorical(encoded_Y)

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
y_test = np_utils.to_categorical(encoded_Y)

In [129]:
# this model only has 10 features
model = Sequential()
model.add(Dense(64, input_dim=10, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train,
                    batch_size=8,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# not sure how to improve

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_162 (Dense)            (None, 64)                704       
_________________________________________________________________
dense_163 (Dense)            (None, 7)                 455       
Total params: 1,159
Trainable params: 1,159
Non-trainable params: 0
_________________________________________________________________
Train on 12096 samples, validate on 3024 samples
Epoch 1/10
12096/12096 [==============================] - 6s 501us/step - loss: 13.9021 - acc: 0.1375 - val_loss: 13.4691 - val_acc: 0.1644
Epoch 2/10
12096/12096 [==============================] - 4s 300us/step - loss: 13.9021 - acc: 0.1375 - val_loss: 13.4691 - val_acc: 0.1644
Epoch 3/10
12096/12096 [==============================] - 4s 295us/step - loss: 13.9021 - acc: 0.1375 - val_loss: 13.4691 - val_acc: 0.1644
Epoch 4/10
12096/12096 [==============================] - 4s 325us/